# SageMaker Endpoint Deployment

In [1]:
import boto3, sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
print(bucket) # the default S3 bucket allocated for the SageMaker session

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker-us-east-1-129708987237


In [8]:
import tarfile, os

tar_file = 'hayao_model.tar.gz'
pretrained_model = 'hayao_model.pth'
helper_file_dir = 'code'
helper_file = 'serve.py'

with tarfile.open(tar_file, 'w:gz') as f:
    t = tarfile.TarInfo(helper_file_dir)
    t.type = tarfile.DIRTYPE
    t.mode = 0o755
    
    f.add(pretrained_model)
    f.add(helper_file, arcname=f"{helper_file_dir}/{helper_file}")
f.close()

In [9]:
# Uploading the model to S3
s3 = boto3.client('s3')

s3.upload_file(
    Filename=tar_file,
    Bucket=bucket,
    Key=os.path.basename(tar_file)
)

model_uri = f"s3://{bucket}/{tar_file}"
print(model_uri)

s3://sagemaker-us-east-1-129708987237/hayao_model.tar.gz


In [10]:
import torch
from sagemaker import get_execution_role

py_version = "py310"
role_arn = get_execution_role()
endpoint_name = "CartoonGAN"
instance_type = "ml.m5.xlarge"
torch_version = torch.__version__

print(py_version)
print(role_arn)
print(endpoint_name)
print(instance_type)
print(torch_version)

py310
arn:aws:iam::129708987237:role/service-role/SageMaker-CloudComputingFinalProj
CartoonGAN
ml.m5.xlarge
2.1.0


In [11]:
from sagemaker.pytorch.model import PyTorchModel

pytorch_model = PyTorchModel(
        model_data=model_uri,
        role=role_arn,
        framework_version=torch_version,
        py_version=py_version,
        entry_point='serve.py',
        source_dir='.'
)

In [6]:
!sudo chmod 777 lost+found

In [12]:
pytorch_model.deploy(
        endpoint_name=endpoint_name,
        instance_type=instance_type,
        initial_instance_count=1
    )

-----!